In [14]:
# Overall process
# 1. A passenger first goes to a ID/boarding-pass check queue and is served by one of the servers
# 2. Then the passenger goes to a personal-check queue to be examined by a personal checker

# Goal - average wait time below 15 minutes

# Facts
# 1. Passengers arrive at the ID/boarding-pass check queue with several servers according to a 
# Poisson distribution with λ1 = 5 per minute (μ1 = 0.2 minutes)
# 2. Each server has exponential service time with mean rate μ2 = 0.75 minutes
# 3. After that, passengers line up at personal-check queues where they go through personal scanner 
# (time is uniformly distributed between 0.5 minutes and 1 minute)

import simpy
import random
import statistics

# Create an empty list for storing each passenger's wait time
wait_times = []

RAND_SEED = 1   # Set seed for reproducibility
NUM_SERVER = 1  # Number of servers
NUM_SCANNER = 1  # Number of scanners
ARRIVAL_RATE = 50  # Number of passengers per minute to simulate very busy airport
T_INTERARRIVAL = 0.02  # Exponential distribution with mean = 0.02 minutes
MEAN_SERVER_TIME = 0.75  # Average minutes a server takes to a passenger's request
SCAN_TIME = [0.5, 1]  # Limits for Uniform distribution of time to scan a passenger
SIM_TIME = 720  # Simulation time in minutes


class Security(object):
    def __init__(
        self,
        env: simpy.core.Environment,
        num_servers,
        num_scanners,
        scan_time,
        mean_server_time,
    ):
        self.env = env
        self.server = simpy.Resource(env, num_servers)
        self.scanner = simpy.Resource(env, num_scanners)
        self.scan_time = scan_time
        self.mean_server_time = mean_server_time

    # Servers as resource
    def server_check_queue(self, passenger):
        """The server serving a passenger process. It takes a ''passenger'' and tries to
        serve with ID/boarding pass request"""
        rand_serve_time = random.expovariate(self.mean_server_time)
        yield self.env.timeout(rand_serve_time)
        print(f"Served a passenger, {passenger} in {round(rand_serve_time, 2)} minutes")
        

    # Personal scanners as resource
    def scan_check_queue(self, passenger):
        """The personal scanner scans a passenger process. It takes a ''passenger'' and tries
        to scan before the ''passenger'' goes through"""
        rand_scan_time = random.uniform(self.scan_time[0], self.scan_time[1])
        yield self.env.timeout(rand_scan_time)
        print(f"Scanned a passenger, {passenger} in {round(rand_scan_time, 2)} minutes")


def go_to_security(env, passenger, security):
    """The server processes each passenger's ID/boarding pass request.

    It starts the serving process and waits for it to finish.

    After the passenger completes the request, he/she goes to personal scanner queue and
    starts the scanning process.

    Then the passenger leaves the system.
    """
    arrival_time = env.now
    print(f"{passenger} arrives at the server at {round(arrival_time, 2)} minutes.")

    # Passenger generates a request to use a server
    with security.server.request() as request:
        # Passenger waits for a server to become available if all are currently in use
        yield request
        print(f"{passenger} arrives at the server at {round(env.now, 2)} minutes.")
        # Passenger uses an available server to complete the given process
        yield env.process(security.server_check_queue(passenger))
        print(f"{passenger} leaves the server at {round(env.now, 2)} minutes.")

    with security.scanner.request() as request:
        # Passenger waits for a scanner to become available if all are currently in use
        yield request
        print(f"{passenger} arrives at the scanner at {round(env.now, 2)} minutes.")
        # Passenger uses an available server to complete the given process
        yield env.process(security.scan_check_queue(passenger))
        print(f"{passenger} leaves the scanner at {round(env.now, 2)} minutes.")

    # Append passenger's wait time
    wait_time = env.now - arrival_time
    print(f"{passenger} waited for a total of {wait_time}")
    wait_times.append(wait_time)


def run_security(env, num_servers, num_scanners, scan_time, mean_server_time):
    # create the security process
    security_line = Security(
        env, num_servers, num_scanners, scan_time, mean_server_time
    )

    for passenger in range(ARRIVAL_RATE):
        env.process(go_to_security(env, passenger, security_line))

    while True:
        yield env.timeout(random.expovariate(T_INTERARRIVAL))

        passenger += 1
        env.process(go_to_security(env, passenger, security_line))


# Calculate the average time a passenger spends from the time of arrival to finishing security check
def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    return average_wait


def main():
    random.seed(RAND_SEED)
    
    # Run the simulation
    env = simpy.Environment()
    env.process(run_security(env, NUM_SERVER, NUM_SCANNER, SCAN_TIME, MEAN_SERVER_TIME))
    env.run(until=SIM_TIME)
    average_time = get_average_wait_time(wait_times)
    print("Running simulation...", f"\nThe average wait time is {average_time}.")

if __name__ == "__main__":
    main()

0 arrives at the server at 0 minutes.
1 arrives at the server at 0 minutes.
2 arrives at the server at 0 minutes.
3 arrives at the server at 0 minutes.
4 arrives at the server at 0 minutes.
5 arrives at the server at 0 minutes.
6 arrives at the server at 0 minutes.
7 arrives at the server at 0 minutes.
8 arrives at the server at 0 minutes.
9 arrives at the server at 0 minutes.
10 arrives at the server at 0 minutes.
11 arrives at the server at 0 minutes.
12 arrives at the server at 0 minutes.
13 arrives at the server at 0 minutes.
14 arrives at the server at 0 minutes.
15 arrives at the server at 0 minutes.
16 arrives at the server at 0 minutes.
17 arrives at the server at 0 minutes.
18 arrives at the server at 0 minutes.
19 arrives at the server at 0 minutes.
20 arrives at the server at 0 minutes.
21 arrives at the server at 0 minutes.
22 arrives at the server at 0 minutes.
23 arrives at the server at 0 minutes.
24 arrives at the server at 0 minutes.
25 arrives at the server at 0 minut